In [1]:
from adapters import AutoAdapterModel, AdapterConfig
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [21]:
#config = AdapterConfig.load("houlsby", non_linearity="gelu", reduction_factor=2)

In [24]:
#model.load_adapter("adapter_chatbot")

In [22]:
#model.adapter_summary()

In [23]:
#model

In [2]:
ruta_del_archivo = 'E:/Git Hub Inteligencia Artificial/Practica Modelos LLM/notebooks/data.xlsx'
dataframe = pd.read_excel(ruta_del_archivo)

# Muestra las primeras filas del DataFrame para verificar
print(dataframe.columns)

Index(['Unnamed: 0', 'Preguntas', 'Respuestas', 'Clasificación'], dtype='object')


In [3]:
dfChatColumns = ['Preguntas','Respuestas']
dfChat = dataframe[dfChatColumns]

In [4]:
dfChat

,Preguntas,Respuestas
0,Hola,"Hola, en que puedo ayudarte?"
1,¿Cuantos programas tienes?,"Indicame que area quisieras:Tecnológicas, Cien..."
2,Que cursos vendes,"Indicame que area quisieras:Tecnológicas, Cien..."
3,Que cursos tienes,"Indicame que area quisieras:Tecnológicas, Cien..."
4,Cursos disponibles,"Que área deseas:Tecnológicas, Ciencias Económi..."
...,...,...
58,Quisiera saber acerca de tus cursos de formación,"Que área deseas:Tecnológicas, Ciencias Económi..."
59,Cuál es la gama de cursos que tienes disponible?,"Indicame que area quisieras:Tecnológicas, Cien..."
60,¿Cuál es la gama de cursos que tienes disponible?,"Que área deseas:Tecnológicas, Ciencias Económi..."
61,Estoy buscando detalles sobre tus programas ed...,"Indicame que area quisieras:Tecnológicas, Cien..."


In [5]:
class DatasetChatbt(Dataset):
    def __init__(self,data,tokenizer, max_length=128):
        self.dataset=data
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return (len(self.dataset))-1
    def __getitem__(self, index):
        input = self.dataset['Preguntas'][index]
        target = self.dataset['Respuestas'][index]
        inpTokn = self.tokenizer.encode(input, max_length=self.max_length,padding="max_length",add_special_tokens=True,truncation=True)
        labelTokn = self.tokenizer.encode(target, max_length=self.max_length,padding="max_length",add_special_tokens=True,truncation=True)
        inpToknR = torch.tensor(inpTokn, dtype=torch.long)
        labelTokn = torch.tensor(labelTokn, dtype=torch.long)
        mask = np.ones(128)
        return {'input_ids':inpToknR,'labels': labelTokn, 'attention_mask':mask}

In [6]:
len(dfChat)

63

In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [8]:
datasetChatbot = DatasetChatbt(dfChat, tokenizer)

In [9]:
for i in datasetChatbot:
    print(i)

{'input_ids': tensor([  101, 20220, 10330,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

KeyError: 63

In [ ]:
#model.load_adapter("es/wiki@ukp", config=config)

In [12]:
model = AutoAdapterModel.from_pretrained("bert-base-multilingual-cased")


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

In [20]:
#model.add_adapter("chatbot")

In [14]:
#model

In [13]:
model.add_causal_lm_head("chatbot", activation_function='gelu')
model.add_adapter("chatbot")
model.train_adapter("chatbot")

In [23]:
model

BertAdapterModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttentionWithAdapters(
              (query): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (key): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (value): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (d

In [15]:
model

BertAdapterModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttentionWithAdapters(
              (query): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (key): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (value): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (d

In [ ]:
#model.adapter_summary()

In [ ]:
#model.add_adapter("es/wiki@up")

In [ ]:
#model.adapter_summary()

In [ ]:
#model.set_active_adapters('es/wiki@up')

In [15]:
#model.adapter_summary()

In [16]:
#model.add_causal_lm_head("chatbot", activation_function='gelu')

In [17]:
model.add_classification_head("recommend_client",num_labels=2)
#The weights of this classification head can be stored together with the adapter weights to allow for a full reproducibility. The method call model.set_active_adapters("sst-2") registers the sst-2 adapter as a default for training. This also supports adapter stacking and adapter fusion!
#model.set_active_adapters("es/wiki@ukp")

In [18]:
model.get_labels_dict('recommend_client')

{0: 'LABEL_0', 1: 'LABEL_1'}

In [20]:
#model.adapter_summary()

In [21]:
#model

In [30]:
model.add_adapter("chatbot")
model.add_causal_lm_head('chatbot', activation_function='gelu')

In [32]:
model

BertAdapterModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttentionWithAdapters(
              (query): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (key): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (value): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (d

In [31]:
from adapters import init
from adapters import ModelAdaptersMixin
ModelAdaptersMixin
init(model)
#model.add_causal_lm_head('chatbot', activation_function='gelu')
model.set_active_adapters("chatbot")
model.train_adapter("chatbot")

In [72]:
from transformers import TrainingArguments
from adapters import AdapterTrainer
training_args = TrainingArguments(
  output_dir="./examples",
  do_train=True,
  remove_unused_columns=False,
  learning_rate=5e-4,
  num_train_epochs=65,
)
trainer = AdapterTrainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        train_dataset=datasetChatbot,
    )



In [73]:
trainer.train() #con 20 reps

Step,Training Loss


TrainOutput(global_step=520, training_loss=0.16259175905814538, metrics={'train_runtime': 3389.0258, 'train_samples_per_second': 1.189, 'train_steps_per_second': 0.153, 'total_flos': 272253553228800.0, 'train_loss': 0.16259175905814538, 'epoch': 65.0})

In [17]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=120, training_loss=1.3425552368164062, metrics={'train_runtime': 944.2773, 'train_samples_per_second': 0.985, 'train_steps_per_second': 0.127, 'total_flos': 62827743052800.0, 'train_loss': 1.3425552368164062, 'epoch': 15.0})

In [18]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=120, training_loss=1.0520692189534506, metrics={'train_runtime': 939.3076, 'train_samples_per_second': 0.99, 'train_steps_per_second': 0.128, 'total_flos': 62827743052800.0, 'train_loss': 1.0520692189534506, 'epoch': 15.0})

In [74]:
model.save_adapter("adapter_chatbot", "chatbot", with_head=True)

In [82]:
tokn= tokenizer.encode("Que cursos tienes", max_length=128,padding="max_length",add_special_tokens=True,truncation=True)

In [5]:
model.add_adapter('recommendation')
model.set_active_adapters("recommendation")
model.train_adapter("recommendation")

In [ ]:
from transformers import TrainingArguments
from adapters import AdapterTrainer, Seq2SeqAdapterTrainer, AdapterConfig
training_args = TrainingArguments(
  output_dir="./examples",
  do_train=True,
  remove_unused_columns=False,
  learning_rate=5e-4,
  num_train_epochs=10,
  
)
trainer = AdapterTrainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        train_dataset=datasetClassification,
    )

In [8]:
for i in model.adapters_config:
    print(i)

recommendation
chatbot


In [18]:
from datasets import Dataset
from transformers import AutoTokenizer

# Supongamos que tienes una lista de textos y sus correspondientes etiquetas
texts = ["Texto 1", "Texto 2", "Texto 3"]
labels = [0, 1, 0]  # 0 o 1 representan las clases

# Crea un diccionario con los datos
data = {"text": texts, "label": labels}

# Crea un objeto Dataset
dataset = Dataset.from_dict(data)

# Descarga el tokenizador preentrenado
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokeniza los textos y añade las etiquetas
#def tokenize_function(examples):
#    return tokenizer(examples["text"], padding=True, truncation=True)
num1 = 0
def prueba(ex):
    num = 3
    return ex + "-"+num
tokenized_datasets = dataset.map(prueba, batched=True)

# Imprime las primeras filas del conjunto de datos tokenizado
print(tokenized_datasets["train"][0])

# Salida esperada:
# {'attention_mask': [1, 1, 1, ...], 'input_ids': [101, ..., 102], 'label': 0}


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

TypeError: unsupported operand type(s) for +: 'LazyBatch' and 'str'

In [19]:
tokenized_datasets

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3
})

In [16]:
for i in tokenized_datasets:
    print(i)

{'text': 'Texto 1', 'label': 0, 'input_ids': [101, 3793, 2080, 1015, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}
{'text': 'Texto 2', 'label': 1, 'input_ids': [101, 3793, 2080, 1016, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}
{'text': 'Texto 3', 'label': 0, 'input_ids': [101, 3793, 2080, 1017, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}


In [75]:
from transformers import BertLMHeadModel, BertTokenizer
from adapters import init
model2 = BertLMHeadModel.from_pretrained("bert-base-multilingual-cased")
tokenizer2 = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
init(model2)
model2.load_adapter("adapter_chatbot")
model2.set_active_adapters("chatbot")


If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Model class 'BertAdapterModel' of found prediction head does not match current model class.


In [77]:
model2

BertLMHeadModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttentionWithAdapters(
              (query): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (key): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (value): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (dr

In [78]:
PREFIX = "Que cursos tienes"
encoding = tokenizer2(PREFIX, return_tensors="pt")
output_sequence = model2(
  input_ids=encoding["input_ids"],
  attention_mask=encoding["attention_mask"],
)

In [15]:
for generated_sequence_idx, generated_sequence in enumerate(output_sequence):
        print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
        generated_sequence = generated_sequence.tolist()

        # Decode text
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
        # Remove EndOfSentence Tokens
        text = text[: text.find(tokenizer.eos_token)]

        print(text)

=== GENERATED SEQUENCE 1 ===


AttributeError: 'str' object has no attribute 'tolist'

In [34]:
import torch
print(tokenizer2.decode(torch.argmax(torch.softmax(output_sequence[0],dim=-1))))

[ U N K ]


In [ ]:
output = model2.generate(
  input_ids=encoding["input_ids"],
  temperature= 0.7,
  do_sample=True,
 # num_return_sequences=5,
  max_length=100
)

In [71]:
tokenizer.decode(output[0])

'[CLS] Que cursos tienes [SEP].................................... de...........,,....,......... a..........,..,...............'

In [15]:
output_sequence.last_hidden_state[0][4]

tensor([ 2.2503e-02,  4.1941e-01,  6.6613e-01, -2.2243e-01,  2.5739e-01,
        -2.7369e-01, -3.0164e-02,  1.5354e-01,  2.1095e-01,  3.8426e-01,
         2.7959e-01,  8.5122e-01, -2.8612e-01,  5.8358e-03, -2.7255e-01,
        -4.7624e-01,  4.5795e-01,  4.8436e-01, -3.6848e-01,  4.3551e-01,
        -7.9584e-01,  5.3310e-01, -3.1700e-02, -2.9465e-01, -3.0209e-01,
        -5.6167e-01, -9.2738e-01, -3.0842e-01,  2.8994e-02, -7.7722e-01,
         8.5338e-01,  1.4951e-01, -4.6942e-01,  6.7728e-01, -5.3971e-01,
        -3.3510e-02,  2.6035e-01,  2.5488e-01,  2.7977e-01, -5.1890e-01,
        -1.3762e-01,  5.9008e-01, -6.4982e-01, -3.5655e-01, -3.7224e-01,
         3.6556e-01, -5.9568e-01, -1.1729e-01,  2.5384e-01, -9.5346e-01,
        -5.4603e-01, -2.9171e-01, -8.8551e-01, -3.1786e-01,  6.8583e-01,
         2.2750e-01, -4.1812e-01, -2.0969e-02,  5.2535e-01, -1.6335e-01,
        -2.6737e-03,  3.4067e-01,  1.1915e-01, -5.7344e-01, -1.8157e-01,
         5.7931e-01,  4.3385e-01, -5.1401e-01,  4.8

In [17]:
output_sequence.last_hidden_state[0,4]

tensor([ 2.2503e-02,  4.1941e-01,  6.6613e-01, -2.2243e-01,  2.5739e-01,
        -2.7369e-01, -3.0164e-02,  1.5354e-01,  2.1095e-01,  3.8426e-01,
         2.7959e-01,  8.5122e-01, -2.8612e-01,  5.8358e-03, -2.7255e-01,
        -4.7624e-01,  4.5795e-01,  4.8436e-01, -3.6848e-01,  4.3551e-01,
        -7.9584e-01,  5.3310e-01, -3.1700e-02, -2.9465e-01, -3.0209e-01,
        -5.6167e-01, -9.2738e-01, -3.0842e-01,  2.8994e-02, -7.7722e-01,
         8.5338e-01,  1.4951e-01, -4.6942e-01,  6.7728e-01, -5.3971e-01,
        -3.3510e-02,  2.6035e-01,  2.5488e-01,  2.7977e-01, -5.1890e-01,
        -1.3762e-01,  5.9008e-01, -6.4982e-01, -3.5655e-01, -3.7224e-01,
         3.6556e-01, -5.9568e-01, -1.1729e-01,  2.5384e-01, -9.5346e-01,
        -5.4603e-01, -2.9171e-01, -8.8551e-01, -3.1786e-01,  6.8583e-01,
         2.2750e-01, -4.1812e-01, -2.0969e-02,  5.2535e-01, -1.6335e-01,
        -2.6737e-03,  3.4067e-01,  1.1915e-01, -5.7344e-01, -1.8157e-01,
         5.7931e-01,  4.3385e-01, -5.1401e-01,  4.8

In [146]:
output_sequence.last_hidden_state[0][0]

tensor([-9.2124e-03,  3.2905e-01,  3.6787e-01, -1.0757e-01,  1.6928e-01,
        -2.8946e-01, -3.6340e-02,  7.3389e-04,  5.5208e-02,  2.3461e-01,
        -1.1046e-01,  3.4048e-01, -3.1462e-01,  2.2572e-01, -5.2292e-01,
        -3.0736e-01,  1.6148e-01,  5.4555e-01, -1.2793e-01,  3.5992e-01,
        -2.9054e-01,  2.6632e-01, -2.7302e-01, -2.6025e-01, -1.6995e-01,
        -5.0727e-01, -4.0929e-01,  1.8389e-01,  5.0150e-01, -3.5916e-01,
         4.6564e-01,  1.7602e-01, -4.2114e-01,  3.7507e-01, -3.2632e-01,
        -2.6592e-01, -1.4109e+00, -1.6923e-01,  1.6739e-01, -5.2055e-01,
        -2.3246e-01,  3.3851e-01, -2.1186e-01, -2.5042e-01, -2.2797e-01,
         1.0526e+00, -2.1193e-01, -7.5878e-02,  1.0902e+00, -5.1219e-01,
        -4.4106e-01, -3.3145e-01, -3.4135e-01, -1.4278e+00,  5.0946e-01,
         1.6275e-01, -2.3829e-01, -7.1904e-02,  2.7914e-01, -7.7504e-02,
        -2.3150e-02,  5.2181e-01,  8.1356e-02, -2.8296e-01, -1.4297e-01,
         3.3649e-01,  2.2748e-01, -3.3528e-01, -7.4

In [54]:
model.adapter_summary()

'================================================================================\nName                     Architecture         #Param      %Param  Active   Train\n--------------------------------------------------------------------------------\nchatbot                  bottleneck          894,528       0.503       1       1\n--------------------------------------------------------------------------------\nFull model                               177,853,440     100.000               1\n================================================================================'

In [55]:
from transformers import TrainingArguments
from adapters import AdapterTrainer
model.set_active_adapters("chatbot")
model.train_adapter("chatbot")
training_args = TrainingArguments(
  output_dir="./examples",
  do_train=True,
  remove_unused_columns=False,
  learning_rate=5e-4,
  num_train_epochs=10,
)
trainer = AdapterTrainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        train_dataset=datasetChatbot,
    )


AttributeError: 'BertModel' object has no attribute 'active_head'

In [ ]:
trainer.train()